In [1]:
# Import packages
import pandas as pd
import numpy as np
import datetime
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_validate, KFold
from sklearn.metrics import accuracy_score, mean_squared_error, cohen_kappa_score, mean_absolute_error, f1_score
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE, SMOTENC
import matplotlib.pyplot as plt
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.decomposition import PCA
from scipy import stats
from math import sqrt



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GaussianNoise
from keras.optimizers import SGD
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

Using TensorFlow backend.


In [2]:
path = "/Users/Stijn/Documents/Master Data Science and Society/Block 3/thesis/code/thesis_Code/"
mood = pd.read_csv(path+'mood_imputed_median.csv', sep = ',', index_col=0)
mood['cheerful_last'] = mood.groupby('user_id')['cheerful'].shift()
mood.loc[(pd.isnull(mood.cheerful_last)), 'cheerful_last'] = mood['cheerful']
mood_cheerful = mood.drop(["bored", "anxious", "content", "user_id", "response_time"], axis=1)

In [3]:
# Normalize predictors
mood_normz = mood_cheerful.copy()
features_to_normalize = ['day_time_window', 'average_TimeUse', 'messaging', 'socialnetworking', 'otherapp']
mood_normz[features_to_normalize] = mood_normz[features_to_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))

In [4]:
# X = predictors, y = target
X = mood_normz.iloc[:,1:]
y = mood_normz.iloc[:, 0:1]

# Convert float to int
y = y.astype(int)

In [5]:
X, y = shuffle(X, y)

In [6]:
X = X.iloc[:,:-1]
X_onlylastmood = X.iloc[:,-1:]

In [7]:
def macroaverage_mae(test, pred):
    mae_0, mae_1, mae_2, mae_3, mae_4, mae_5 = (0,0,0,0,0,0)

    for i in range(len(pred)):
        if test[i] == 0 and pred[i] != 0:
            mae_0 += (abs(test[i]-pred[i]))
        if test[i] == 1 and pred[i] != 1:
            mae_1 += (abs(test[i]-pred[i]))
        if test[i] == 2 and pred[i] != 2:
            mae_2 += (abs(test[i]-pred[i]))
        if test[i] == 3 and pred[i] != 3:
            mae_3 += (abs(test[i]-pred[i]))
        if test[i] == 4 and pred[i] != 4:
            mae_4 += (abs(test[i]-pred[i]))
        if test[i] == 5 and pred[i] != 5:
            mae_5 += (abs(test[i]-pred[i]))

    cnt_0, cnt_1, cnt_2, cnt_3, cnt_4, cnt_5 = (0,0,0,0,0,0)
    for z in range(len(test)):
        if test[z] == 0:
            cnt_0 += 1
        if test[z] == 1:
            cnt_1 += 1
        if test[z] == 2:
            cnt_2 += 1
        if test[z] == 3:
            cnt_3 += 1
        if test[z] == 4:
            cnt_4 += 1
        if test[z] == 5:
            cnt_5 += 1

    mae_macroaverage = ((mae_0/cnt_0) + (mae_1/cnt_1) + (mae_2/cnt_2) + (mae_3/cnt_3) + (mae_4/cnt_4) + (mae_5/cnt_5)) / 6
    return mae_macroaverage 

In [8]:
y[:5]

,cheerful
13121,4
2354,3
11339,1
13020,3
7884,4


In [9]:
n_split=10
f1_scores = []
mmae_scores = []
f1_scores_classes = []
for train_index, test_index in KFold(n_splits = n_split, random_state=2, shuffle=True).split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    sm = SMOTENC(categorical_features=[0, 2], random_state = 2) 
    X_train_oversampl, y_train_oversampl = sm.fit_sample(X_train, y_train['cheerful'].ravel()) 
    
    y_train_df = pd.DataFrame(y_train_oversampl)

    ohe = OneHotEncoder(sparse=False, categories='auto')
    y_onehot = ohe.fit_transform(y_train_df)
    
    X_use, y_use = shuffle(X_train_oversampl, y_onehot)
    X_use = pd.DataFrame(X_use)
    y_use = pd.DataFrame(y_use)
    
    model = Sequential()
    model.add(Dense(24, input_dim=6, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(6, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_use, y_use,epochs=100, batch_size=512, verbose=0)
    y_pred = model.predict(X_test)
    test = y_test.values

    pred = list()
    for i in range(len(y_pred)):
        pred.append(np.argmax(y_pred[i]))

    
    f1_scores_classes.append(f1_score(test, pred, average=None))
    f1_scores.append(round(f1_score(test, pred, average='weighted'), 4))
    mmae_scores.append(macroaverage_mae(test, pred))
    

In [10]:
print("Macro-averaged F1-scores: ", f1_scores)
print("Mean MA F1-scores: ", round(np.mean(f1_scores), 4))
print("Std MA F1-scores: ", round(np.std(f1_scores), 4))

Macro-averaged F1-scores:  [0.1225, 0.0955, 0.1375, 0.1091, 0.0842, 0.1089, 0.1325, 0.1236, 0.1509, 0.1237]
Mean MA F1-scores:  0.1188
Std MA F1-scores:  0.0189


In [11]:
mmae_scores_rounded = []
for a in mmae_scores:
    for b in a:
        mmae_scores_rounded.append(round(b, 4))
        
print("Macro-averaged MAE: ", mmae_scores_rounded)
print("Mean MMAE-scores: ", round(np.mean(mmae_scores), 4))
print("Std MMAE-scores: ", round(np.std(mmae_scores), 4))

Macro-averaged MAE:  [2.14, 2.2322, 2.0902, 2.0799, 2.2143, 2.2477, 1.898, 2.0762, 1.9595, 2.0675]
Mean MMAE-scores:  2.1006
Std MMAE-scores:  0.108


In [12]:
print("Mean F1-score per class: ", [round(np.mean(x), 4) for x in zip(*f1_scores_classes)])
print("STD F1-score per class: ", [round(np.std(x), 4) for x in zip(*f1_scores_classes)])

Mean F1-score per class:  [0.2029, 0.1637, 0.0391, 0.1396, 0.0893, 0.143]
STD F1-score per class:  [0.0272, 0.0631, 0.0457, 0.0409, 0.0348, 0.0156]
